In [15]:
import re
from pprint import pprint as pp
def inlines(lines):
    tags = re.compile('^@(image|box|quote):')
    return (line for line in map(str.strip, lines) if tags.match(line))

def add_text_tag(lines):
    return (line if line.startswith('@') else f'@txt:{line}' for line in (l.strip() for l in lines) if line)

def links_to_dict(story):
    return [{'label': l.number, 'url': l.link} for l in story.inline_links.all()]

def images_to_dict(story):
    images = (im.child for im in story.images().order_by('index'))
    return [{
        'index': im.index,
        'id': im.imagefile.pk, 
        'caption': im.caption,
        'creditline': im.creditline,
        'ratio': im.aspect_ratio,
    } for im in images]

def videos_to_dict(story):
    videos = (vd.child for vd in story.videos().order_by('index'))
    return [{
        'index': vd.index,
        'link': vd.link,
        'caption': vd.caption,
        'creditline': vd.creditline,
        'ratio': vd.aspect_ratio,
    } for vd in videos]


In [16]:
for vid in StoryVideo.objects.all():
    print(vid.link)

https://vimeo.com/125951244
https://vimeo.com/125978502
https://vimeo.com/109002468
https://vimeo.com/109002178
https://vimeo.com/128929942
https://vimeo.com/128929943
https://vimeo.com/121352681
https://vimeo.com/121352681
https://vimeo.com/147010353
https://www.youtube.com/watch?v=lAJ5KJoC18s/
https://www.youtube.com/watch?v=E26jZB81j0Y/
https://www.youtube.com/watch?v=o4mI9MZsQF0/
https://www.youtube.com/watch?v=R_V_pVQCb4s/
https://www.youtube.com/watch?v=akLs-oX2mDM/
https://www.youtube.com/watch?v=akLs-oX2mDM/
https://vimeo.com/216760518


In [18]:
def serialize_story(story):
    title = f'@tit:{story.title}' if story.title else ''
    kicker = f'@stikktit:{story.kicker}' if story.kicker else ''
    lede = f'@ing:{story.lede}' if story.lede else ''
    theme = f'@tema:{story.theme_word}' if story.theme_word else ''
    bylines = (str(bl) for bl in story.byline_set.all())
    pullquotes = (pq.pullquote.bodytext_markup for pq in story.pullquotes())
    asides = (aside.aside.bodytext_markup for aside in story.asides())
    images = list
    body = story.bodytext_markup.splitlines()
    body = add_text_tag(body)
    lines = '\n'.join([kicker, title, theme, lede, *body, *bylines, *asides, *pullquotes]).splitlines()
    return {
        'pk': story.pk,
        'text': [line for line in lines if line],
        'links': links_to_dict(story),
        'videos': videos_to_dict(story),
        'images': images_to_dict(story),
    }
  
for story in Story.objects.exclude(inline_links=None).order_by('?')[:3]:
    pp(serialize_story(story))

{'images': [{'caption': 'Complaining his way down: Jostein Hult went from a B '
                        'to a fail when he complained on his grade. – The '
                        'grading seems random when the deviations can be so '
                        'large, he says.\u2028'
                        '\u2028',
             'creditline': '',
             'id': 30193,
             'index': 1,
             'ratio': 0.0},
            {'caption': 'Complained his way to the top: Karl Kristian R. '
                        'Kirchhoff went from a D to an A, when he complained '
                        'on his grade.\u2028'
                        '\u2028',
             'creditline': '',
             'id': 30194,
             'index': 2,
             'ratio': 0.0}],
 'links': [{'label': 1, 'url': 'http://oldwww.universitas.no/news/'}],
 'pk': 60304,
 'text': ['@tit:From B to F',
          '@ing:While one student went from B to F, another went from D to an '
          'A after complaining on 

In [ ]:
story = Story.objects.get(pk=53597)
serialize_story(story)

In [ ]:
list(inlines(story.bodytext_markup.splitlines()))


In [ ]:
[im.storyimage for im in story.images()]

In [ ]:
[(el.index, el.child) for el in story.storyelement_set.all()]

In [ ]:
story = Story.objects.exclude(inline_links=None).order_by('?').first()
list(story.inline_links.all())

